In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mlp import Layer, MultilayerPerceptron, SquaredError, Relu, Linear
from sklearn.metrics import r2_score

# -------------------- Loading MPG Dataset -------------------- #
class MPGDataLoader:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path

    def load_data(self):
        columns = ["mpg", "cylinders", "displacement", "horsepower", "weight",
                   "acceleration", "model year", "origin", "car name"]
        df = pd.read_csv(self.dataset_path, sep='\s+', names=columns, na_values="?")
        df.dropna(inplace=True)
        df.drop(columns=["car name"], inplace=True, errors='ignore')
        df["horsepower"] = pd.to_numeric(df["horsepower"], errors='coerce')
        df.dropna(inplace=True)

        y = df["mpg"].values.reshape(-1, 1)
        X = df.drop(columns=["mpg"]).values
        return X, y

# Load dataset
dataset_path = 'mpg-data/auto-mpg.data'
dataloader = MPGDataLoader(dataset_path)
X, y = dataloader.load_data()

# -------------------- Data Preprocessing -------------------- #
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train)
X_val = scaler_x.transform(X_val)
X_test = scaler_x.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# -------------------- Hyperparameter Tuning: Learning Rate -------------------- #
input_dim = X_train.shape[1]

learning_rates = [0.0001, 0.0005, 0.001, 0.01, 0.1]  # Different learning rates to test

results = {}

for lr in learning_rates:
    print(f"\nTraining model with learning rate = {lr}...")

    layers = [
        Layer(input_dim, 64, Relu(), dropout_rate=0.2),
        Layer(64, 64, Relu(), dropout_rate=0.2),
        Layer(64, 64, Relu(), dropout_rate=0.2),
        Layer(64, 1, Linear())
    ]

    mlp = MultilayerPerceptron(layers)
    train_losses, val_losses = mlp.train(
        X_train, y_train, X_val, y_val,
        SquaredError(), learning_rate=lr, batch_size=32, epochs=150, optimizer='vanilla', momentum=0.7
    )

    y_pred = mlp.forward(X_test, training=False)

    # Ensure y_pred is reshaped correctly before inverse transformation
    y_pred = y_pred.reshape(-1, 1)
    y_pred = scaler_y.inverse_transform(y_pred)

    # Also inverse-transform y_test for accurate comparison
    y_test_original = scaler_y.inverse_transform(y_test)

    # Compute R² Score
    r2 = r2_score(y_test_original, y_pred)

    # Store results
    results[lr] = round(r2 * 100, 2)  # Convert to percentage
    print(f"R² Score for learning rate {lr}: {results[lr]}%")

# Identify Best Learning Rate
best_lr = max(results, key=results.get)
best_r2 = results[best_lr]

# -------------------- Summary of Results -------------------- #
print("\n===== Summary of Runs =====")
for lr, r2 in results.items():
    print(f"R² value for learning rate {lr}: {r2}%")

print(f"\nBest Learning Rate: {best_lr} with R² = {best_r2}%")


Training model with learning rate = 0.0001...
Epoch 1/150 - Train Loss: 1.1221, Val Loss: 1.0421
Epoch 2/150 - Train Loss: 1.1556, Val Loss: 0.9632
Epoch 3/150 - Train Loss: 0.9957, Val Loss: 0.8469
Epoch 4/150 - Train Loss: 1.0460, Val Loss: 0.8696
Epoch 5/150 - Train Loss: 1.0289, Val Loss: 0.9113
Epoch 6/150 - Train Loss: 0.9795, Val Loss: 0.9371
Epoch 7/150 - Train Loss: 0.9741, Val Loss: 0.8671
Epoch 8/150 - Train Loss: 0.9253, Val Loss: 0.7237
Epoch 9/150 - Train Loss: 0.9803, Val Loss: 0.7600
Epoch 10/150 - Train Loss: 0.9070, Val Loss: 0.8514
Epoch 11/150 - Train Loss: 0.8961, Val Loss: 0.7562
Epoch 12/150 - Train Loss: 0.9012, Val Loss: 0.7772
Epoch 13/150 - Train Loss: 0.8946, Val Loss: 0.6558
Epoch 14/150 - Train Loss: 0.8538, Val Loss: 0.7506
Epoch 15/150 - Train Loss: 0.8517, Val Loss: 0.7462
Epoch 16/150 - Train Loss: 0.8166, Val Loss: 0.6747
Epoch 17/150 - Train Loss: 0.8399, Val Loss: 0.6439
Epoch 18/150 - Train Loss: 0.7800, Val Loss: 0.6122
Epoch 19/150 - Train Loss: